In [6]:
import requests
import json
from bs4 import BeautifulSoup
import re

In [7]:
class Product:
    def __init__(self):
        self.id = ""
        self.url = ""
        self.title = ""
        self.price = 0
        self.stock_state = ""
        self.last_update = 0

    def __repr__(self):
        return f"id:{self.id}\nname:{self.title}\nurl:{self.url}\nprice:{self.price}\nstock:{self.stock_state}\nupdate:{self.last_update}"

Get boxes from endpoint

In [8]:
def getBoxes():
    headers = {
        'authority': 'www.alza.cz',
        'dnt': '1',
        'accept-language': 'cs-CZ',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
        'content-type': 'application/json; charset=UTF-8',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'cache-control': 'no-cache',
        'x-requested-with': 'XMLHttpRequest',
        'request-id': '|a997512e379f4357a649b54cb75c3b15.3fb610ee47e0443e',
        'origin': 'https://www.alza.cz',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm',
        'cookie': '__uzma=13b1dde2-65a0-4ed1-b157-ef44695b964a; __uzmb=1602681498; __uzme=1991; lb_id=3dff50518ae596801084d72f0631899f; VZTX=2736279313; CCC=18863907; CriticalCSS=6858194; .AspNetCore.Culture=c%3Dcs-CZ%7Cuic%3Dcs-CZ; ai_user=gTrHEsE2aCBUi9cIZo0029|2020-10-14T13:18:17.264Z; _vwo_uuid_v2=D5BB76F154E337D53FC2D31085DC2FAF2|3cdf93c7b9e3c04746c668405e65a40c; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmbj2=1602681499; __uzmaj2=ca7c7cba-1334-4874-854c-8c618c4bfbe6; _gid=GA1.2.1169509407.1602681499; _gcl_au=1.1.394470555.1602681500; _fbp=fb.1.1602681501926.1641455788; db_ui=214d939e-31ac-32c8-2152-b8247fd79061; _hjid=ae43759a-6eb8-4351-8b53-c72061a366f1; db_uicd=8d4d7096-845d-9a67-60b3-6a23504dae96; SL_C_23361dd035530_KEY=b0375d591e85b0affec1d581bfed3c760cb8c56d; SL_C_23361dd035530_VID=Gk0LFsie0Hn; SL_C_23361dd035530_SID=8fTmbpNkJsm; TPL=1; PVCFLP=6; ai_session=f72wAyPMQH3r7oqKe/hcsu|1602681497653|1602684437193; __uzmdj2=1602684438; __uzmcj2=606872525505; i18next=cs-CZ; _gat=1; __uzmd=1602684439; __uzmc=844786736559; _gat_UA-948269-48=1; _dc_gtm_UA-948269-48=1; _ga=GA1.1.1545458139.1602681499; _ga_FGLGFS7LP0=GS1.1.1602684342.2.1.1602684481.17; sc/bezzrcadlovky-bez-objektivu/18863907.htm=5500',
    }

    data = '{"idCategory":18863907,"producers":"","parameters":[],"idPrefix":0,"prefixType":0,"page":1,"pageTo":4,"inStock":false,"newsOnly":false,"commodityStatusType":null,"upperDescriptionStatus":0,"branchId":-2,"sort":0,"categoryType":1,"searchTerm":"","sendProducers":false,"layout":0,"append":false,"leasingCatId":null,"yearFrom":null,"yearTo":null,"artistId":null,"minPrice":-1,"maxPrice":-1,"shouldDisplayVirtooal":false,"callFromParametrizationDialog":false,"commodityWearType":null,"scroll":5500,"hash":"#f&cst=null&cud=0&pg=1-2&prod=","counter":1}'

    r = requests.post('https://www.alza.cz/Services/EShopService.svc/Filter', headers=headers, data=data)

    if(not r.ok):
        r.raise_for_status()
    else:
        return r.json()['d']['Boxes']

In [30]:
def getProductsDataFromBoxes(boxes):
    """
    Parameters
    ----------
    boxes: str
        html with products from alza
    
    Returns
    -------
    map
        map iterable of tuples containing data for every product in this order: (id, title, url, price, stock state)
    """
    soup = BeautifulSoup(boxes, 'html.parser')
    products = soup.find_all("div", attrs={"class":"fb"})

    def getData(i):
        link = i.find("a", attrs={"data-impression-id":True})

        prodId = link.get('data-impression-id')
        prodTitle = link.get('data-impression-name')
        prodUrl = link.get('href')

        bottom = i.parent.parent.find("div", attrs={"class":"bottom"})
        prodPrice = re.sub(r'\D', '', bottom.find("span", attrs={"class":"c2"}).get_text()) #assuming no decimal values (reasonable in czk)
        prodStock = i.parent.parent.find("div", attrs={"class":"bottom"}).find("div", attrs={"class":["avl","avl extended"]}).find("span").get_text()

        # r2 = requests.get(f"https://www.alza.sk{prodUrl}")
        # if not r2.ok:
        #     r2.raise_for_status()

        # soup2 = BeautifulSoup(r2.content, "html.parser")
        # prodDetail = soup2.find("div", attrs={"class":"detail-page articleLab"})
        # scriptText = prodDetail.find('script').get_text()
        # #scriptText = soup2.find("script", attr={"type":"text/javascript"}).get_text()
        # data = re.search(r'var dataLayer = (\[.*?\]);',scriptText).group(1)
        # j = json.loads(data)

        # prodPrice = j[0]["itemPrice"]

        return (prodId, prodTitle, prodUrl, prodPrice, prodStock)
    
    return map(getData, products)

In [38]:
from datetime import datetime

boxes = getBoxes()
data = getProductsDataFromBoxes(boxes)
# datetimeGenerator = lambda : yield datetime.now()

for i, d in enumerate(zip(data, lambda : yield datetime.now())):
    if(i == 3):
        break
    else:
        print(d)

SyntaxError: invalid syntax (<ipython-input-38-d0f355289715>, line 7)